In [1]:
import sys
from pathlib import Path
import import_ipynb
import numpy as np
import gmsh
from scipy.sparse.linalg import spsolve
from analytical_solution import get_sources, potential_and_gradient

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    from pre_processing import domain, read_mesh
    from fem_processing import boundary_conditions
    from pos_processing import graph_results, error_evaluation
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!


# Project 1: Poisson Problem in a rectangular domain

Considere um problema de _Poisson_ 2D em $\Omega = [0,1]^2$

$$
-\nabla \cdot \left( \nabla u(x,y) \right) = f(x,y)
\tag{1}
$$

onde,
$$
f(x,y) = 2\pi^2 \, sin(\pi x) \, sin(\pi y)
\tag{2}
$$

cujas condições de contorno são $u=0$ em $\partial \Omega$.

A solução analítica deste problema é

$$
u(x,y) = sin(\pi x) \, sin(\pi y)
\tag{3}
$$

Implemente um programa de elementos finitos para resolver o Problema de Valor de Contorno $(1)$.   

Usar malhas com densidades diferentes e verificar a convergência da sua solução à medida que refinna a malha. Calcule os erros na norma $L_2$ e na norma de energia. A melhor forma de verificar a convergência é traçar gráfios em escala _log-log_ da norma do erro em função de h. Se tudo estiver correto, em escala _log-log_ você obterá retas cujas inclinações fornecerão as taxas de convergência.

# `setTypeProblem()`

In [2]:
def setTypeProblem(element_type, lc):
    mesh_data = {}
    
    # Create mesh using gmsh
    gmsh.initialize()
    BOUNDARY = [
        {'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'entire_boundary'}]
    MATERIAL = [
        {'type': 'material_constant_a', 'tag': 201, 'value': 1, 'name': 'entire_domain'}]
    domain.get_rectangular_geometry(
        element_type, BOUNDARY, lc, auto_save=False, view_mesh=False)
    
    # Create mesh Structure Data from gmsh
    mesh_data['nodes_data'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['conn_data'] = read_mesh.get_conn_data(MATERIAL)
    gmsh.finalize()    

    # Get FEM solution
    rho = get_sources(mesh_data)
    Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, rho, element_type)
    ur = spsolve(Agr.tocsr(), fgr.toarray())
    u = boundary_conditions.global_potential_vector(mesh_data, ur)
    return u, mesh_data

# Convergence Analysis

In [3]:
element_types = [('Triangle', 1), ('Triangle', 2), ('Quadrangle', 1), ('Quadrangle', 2)]
analytical_solution = lambda x, y: potential_and_gradient(x, y)
mesh_sizes = [1 / (2**i) for i in range(2, 5)]

# Inicialização dos dicionários para armazenar erros acumulados para cada tipo de elemento
errors_l2 = {et: [] for et in element_types}
errors_energy = {et: [] for et in element_types}

for lc in mesh_sizes:  
    for element_type in element_types:  
        # Resolve a solução numérica uh para o tipo de elemento e refinamento de malha
        uh, mesh_data = setTypeProblem(element_type, lc)  
        
        # Calcula o erro L2, o erro de energia e a energia total
        l2, energy, _ = error_evaluation.get_errors_and_energy(
            mesh_data, element_type, uh, analytical_solution)
        
        # Acumula os erros nos dicionários correspondentes
        errors_l2[element_type].append(l2)
        errors_energy[element_type].append(energy)

# Graphics

In [4]:
# Plotagem do gráfico log-log do erro em função do tamanho da malha para cada ElementType
graph_results.plot_convergence_with_slope(mesh_sizes, errors_l2, model='rectangular', name='L2')
graph_results.plot_convergence_with_slope(mesh_sizes, errors_energy, model='rectangular', name='Energy')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\1_rectangular_domain\pos_processing\pictures\rectangular_convergence_L2.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\1_rectangular_domain\pos_processing\pictures\rectangular_convergence_Energy.svg


### Figure 1: Error Convergence for $L_2$ norm
<figure>
    <img src="pos_processing/pictures/rectangular_convergence_L2.svg" alt="Fig.1" style="width:90%;" />
    <figcaption>Figure 1: Convergence of the numerical solution for $L_2$ norm of the error</figcaption>
</figure>

### Figure 2: Error Convergence for _Energy_ norm
<figure>
    <img src="pos_processing/pictures/rectangular_convergence_Energy.svg" alt="Fig.2" style="width:90%;" />
    <figcaption>Figure 2: Convergence of the numerical solution for Energy norm of the error</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.